In [ ]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import xarray as xr
import rioxarray as rxr
import geopandas as gpd
from shapely.geometry import mapping
import pyproj

C:\Users\Quentin\anaconda3\envs\wildfire\lib\site-packages\xarray\backends\cfgrib_.py:29: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(
C:\Users\Quentin\anaconda3\envs\wildfire\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.12.1 when it was built against 1.12.2, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [26]:
pyproj.datadir.get_data_dir()

'C:\\Users\\Quentin\\anaconda3\\envs\\wildfire\\lib\\site-packages\\ecmwflibs\\share\\proj'

In [39]:
pyproj.datadir.set_data_dir('C:\\Users\\Quentin\\anaconda3\\envs\\wildfire\\lib\\site-packages\\ecmwflibs\\share\\proj\\proj.db')

In [40]:
# Create a path to the data directory
path_data = "../data/Raw/"

In [41]:
# Open the data
ndvi = ndvi = xr.open_dataset(path_data +'Raw_NDVI_16D_1km.nc')

In [42]:
# Open the data
era = xr.open_dataset(path_data +'Raw_weather_4H_9km.nc')

In [43]:
# Print the CRS of the data
print(ndvi.rio.crs)
print(era.rio.crs)

None
None


In [44]:
era

<xarray.Dataset>
Dimensions:    (longitude: 36, latitude: 21, time: 22975)
Coordinates:
  * longitude  (longitude) float32 4.0 4.1 4.2 4.3 4.4 ... 7.1 7.2 7.3 7.4 7.5
  * latitude   (latitude) float32 45.0 44.9 44.8 44.7 ... 43.3 43.2 43.1 43.0
  * time       (time) datetime64[ns] 2010-01-01T04:00:00 ... 2022-07-31T20:00:00
Data variables:
    u10        (time, latitude, longitude) float32 ...
    v10        (time, latitude, longitude) float32 ...
    t2m        (time, latitude, longitude) float32 ...
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-09-15 13:57:35 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [45]:
# Definition of a function to define a crs for the dataray
def define_crs(dataray, crs=2154):
    """Defines a crs for the dataray"""
    return dataray.rio.write_crs(crs, inplace=True)

In [46]:
# Era datasets is in WG84 or EPSG:4326
era = define_crs(era, 4326)

CRSError: Invalid projection: epsg:4326: (Internal Proj Error: proj_create: SQLite error on SELECT name, type, coordinate_system_auth_name, coordinate_system_code, datum_auth_name, datum_code, area_of_use_auth_name, area_of_use_code, text_definition, deprecated FROM geodetic_crs WHERE auth_name = ? AND code = ?: no such column: area_of_use_auth_name)

In [14]:
era_daily = xr.open_dataset(path_data +'era_daily.nc')

In [15]:
era_daily

<xarray.Dataset>
Dimensions:    (longitude: 36, latitude: 21, time: 4595)
Coordinates:
  * longitude  (longitude) float32 4.0 4.1 4.2 4.3 4.4 ... 7.1 7.2 7.3 7.4 7.5
  * latitude   (latitude) float32 45.0 44.9 44.8 44.7 ... 43.3 43.2 43.1 43.0
  * time       (time) datetime64[ns] 2010-01-01 2010-01-02 ... 2022-07-31
Data variables:
    u10        (time, latitude, longitude) float64 ...
    v10        (time, latitude, longitude) float64 ...
    t2m        (time, latitude, longitude) float64 ...
    tp         (time, latitude, longitude) float64 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-09-15 13:57:35 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [17]:
print(era_daily.rio.crs)

None
